# Geophysical particle filter

The point of this notebook is to recreate on an unclassified dataset and unrelated codebase the deep water bathymetric particle filter results, and include additional results for gravity and magnetic measurements. The function of this notebook is to serve as a prototyping and low-scale 'simulation' environment. 

First, let's import the stuff needed.

In [1]:
import numpy as np
from gmt_tool import get_map_section, inflate_bounds, get_map_point
from haversine import haversine, Unit
from tools import load_trackline_data
from matplotlib import pyplot as plt
from particle_filter import run_particle_filter, propagate, plot_error, plot_estimate, plot_map_and_trajectory
import pandas as pd
from datetime import timedelta
from scipy.io import savemat

First we need to tune the particle filter propagation noise to be similar to that of a marine-grade inertial navigation system. A low-end marine-grade INS should have a drift of 1 nm per 24 hours.

In [ ]:
from particle_filter import rmse

time = 24*60 # minutes
noise = np.array([0, 0.1 * np.sqrt(1/60),0])
bound = 1852 # meters
v = 4*bound / 3600 # m / min
P = np.asarray([[0,0,0,0,v,0]])
T = P
err = rmse(P[0,:2], T[0, :2])
errors = [err]
t = 0
u = [0,v,0]

while t < time:    
    P = propagate(P, u, noise=noise)
    T = propagate(T, u, noise=[0,0,0])
    errors.append(rmse(P[0,:2], T[0, :2]))

plt.plot(errors)

In [ ]:
P = propagate(P, [0,v,0], 60)

We'll first do some general examination of the data. Namely, investigating the sensor measurements to see if we can build a reasonable sensor model.

In [ ]:
d_bathy = np.asarray([])
#d_grav = np.asarray([])
#d_mag = np.asarray([])
for i in range(20):
    data = load_trackline_data(f'./data/processed/marine_trackline_{i}.csv')
    min_lon = data.LON.min()
    max_lon = data.LON.max()
    min_lat = data.LAT.min()
    max_lat = data.LAT.max()
    min_lon, min_lat, max_lon, max_lat = inflate_bounds(min_lon, min_lat, max_lon, max_lat, 0.25)
    bathy_map = get_map_section(min_lon, max_lon, min_lat, max_lat, 'relief', '15s', f'track{i}')
#    grav_map = get_map_section(min_lon, max_lon, min_lat, max_lat, 'gravity', '01m', f'track{i}')
#    mag_map = get_map_section(min_lon, max_lon, min_lat, max_lat, 'magnetic', '02m', f'track{i}')
    d_bathy = np.hstack([d_bathy, data['CORR_DEPTH'] - (-get_map_point(bathy_map, data.LON, data.LAT))])
#    d_grav = np.hstack([data['FREEAIR'] - get_map_point(grav_map, data.LON, data.LAT)])
#    d_mag = np.hstack([data['CORR_DEPTH'] - get_map_point(bathy_map, data.LON, data.LAT)])

bathy_sigma = np.std(d_bathy, where=~np.isnan(d_bathy))

Now let's run the trajectory

In [ ]:
# Load the trajectory file
for num in range(20):
    name = f'marine_trackline_{num}'
    data = load_trackline_data(f'./data/processed/{name}.csv')

    # Create the map
    min_lon = data.LON.min()
    max_lon = data.LON.max()
    min_lat = data.LAT.min()
    max_lat = data.LAT.max()
    min_lon, min_lat, max_lon, max_lat = inflate_bounds(min_lon, min_lat, max_lon, max_lat, 0.25)
    geo_map = get_map_section(min_lon, max_lon, min_lat, max_lat, 'relief', '15s', f'track{num}')
    
    # Run the particle filter
    N = 10000
    # Intial parameters
    mu = [data.LAT[0], data.LON[0], 0, 0, 0, 0]
    cov = np.diag([1/60, 1/60, 0, 0.1, 0.1, 0])
    # Run PF
    estimate, error, rms_error = run_particle_filter(mu, cov, N, data, geo_map, measurement_sigma=bathy_sigma)

    

    
    plt.show()
    plt.savefig(f'{name}_Error.png')

    # Save particle filter error characteristics.
    results = {'estimate':estimate, 'error':error, 'rmse':rms_error}
    savemat(f'{name}_errors.mat', results)

In [14]:
from datetime import timedelta

In [27]:
track = pd.read_csv('./data/processed/noaa_tracklines/ew0114.csv', index_col=0)
track

,LAT,LON,BAT_TTIME,CORR_DEPTH,MAG_TOT,MAG_RES,FREEAIR
2001-12-09 05:43:00+05:00,-38.01339,109.88378,6.075,4562.0,60710.0,-6.0,15.4
2001-12-09 05:44:00+05:00,-38.01561,109.88164,6.057,4549.0,60708.0,-8.0,12.9
2001-12-09 05:45:00+05:00,-38.01783,109.87948,6.080,4566.0,60708.0,-8.0,13.1
2001-12-09 05:46:00+05:00,-38.01999,109.87737,6.067,4556.0,60710.0,-8.0,11.5
2001-12-09 05:47:00+05:00,-38.02222,109.87526,6.072,4560.0,60711.0,-7.0,11.5
...,...,...,...,...,...,...,...
2002-01-23 11:30:00+05:00,-46.37401,138.81362,5.244,3925.0,64322.0,-120.0,34.0
2002-01-23 11:40:00+05:00,-46.37278,138.81795,5.225,3911.0,64323.0,-118.0,31.9
2002-01-23 11:50:00+05:00,-46.37151,138.82229,5.207,3897.0,64324.0,-116.0,32.4
2002-01-23 11:10:00+05:00,-46.36555,138.84374,5.333,3993.0,64351.0,-86.0,33.0


In [24]:
from process_dataset import m77t_to_csv

In [26]:
processed = m77t_to_csv(track)

KeyError: ['TIME']